<a href="https://colab.research.google.com/github/krmonline/Sentiment-Analysis-in-Social-Networks/blob/master/FacebookPageAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install facebook-sdk

In [0]:
import facebook
import json
import pandas as pd

In [0]:
graph = facebook.GraphAPI(access_token="EAADf23dcOQgBALfp8KWeZBJEcdHikkRWSv6Wxn18vuA0uwrDc39kEvlxRbH6IBmngNJnCqWFDfdafvFQY4ZCloW8oVPVgoZBC8aJKOLZBRlgbbDZB0uqwwAzM1ikHWX2w7YEcdMpLkSiVHs6O8LEwieq3ZAOyJ5opoAzBjKVtmWTZAC57hiIkAFXSDyJ69s69zTlze7QnXo5AZDZD")

In [7]:
loop_count = 0
df_comments = pd.DataFrame(columns=['id','from','message'])
df = pd.DataFrame(columns=['id','created_time','message','comments'])
js = graph.get_object(id='softnixtech/posts',fields='message,story,created_time,id,comments')
countloop = 0
while('paging' in js):
  countloop = countloop + 1
  if countloop > 5:
    break
  loop_count = loop_count+1
  print(loop_count,end=" ")
  if 'next' in js['paging']:
    arr_page = {}
    for kv in js['paging']['next'].split("?")[1].split("&"):
      k,v = kv.split("=")
      arr_page[k] = v
    after = arr_page['after']
    js = graph.get_object(id='softnixtech/posts',fields='message,story,created_time,id,comments',after=after)
    dftmp = pd.DataFrame(js['data'],columns=['id','created_time','message','comments'])

    df = pd.concat([df,dftmp],ignore_index=True)
df

1 2 3 4 5 

,id,created_time,message,comments
0,162423635476_441168406599447,2019-11-29T03:11:03+0000,Deep Talk ตอนที่ 1 ว่าด้วยเรื่อง การนำ Machine...,NaN
1,162423635476_10157768232795477,2019-11-29T01:22:33+0000,Deep Talk ตอนที่ 1 ว่าด้วยเรื่องการนำ Machine ...,NaN
2,162423635476_2627961090614054,2019-11-28T04:39:10+0000,แนะนำการนำศาสตร์ของการวิเคราะห์ข้อมูลบิ๊กดาต้า...,NaN
3,162423635476_10157763031545477,2019-11-27T06:11:10+0000,แนะนำบูธ Softnix ที่งาน CDIC 2019,NaN
4,162423635476_10157760612950477,2019-11-26T12:39:35+0000,ผ่านไปแล้วกับงาน CDIC 2019 วันแรก พรุ่งนี้ยังม...,{'data': [{'created_time': '2019-11-26T12:48:4...
...,...,...,...,...
120,162423635476_10156644130255477,2018-09-11T03:43:52+0000,Data Governance กับ โครงการ Big Data มันสำคัญอ...,NaN
121,162423635476_10156643881250477,2018-09-11T01:14:09+0000,อับเดทเนื้อหาบทความ ความรู้เกี่ยวกับ Microserv...,NaN
122,162423635476_10156632456500477,2018-09-06T09:00:31+0000,วันนี้ทีมงาน Softnix มีบทความด้าน Data Analyti...,NaN
123,162423635476_10156630341570477,2018-09-05T10:42:07+0000,ขอแจ้งประชาสัมพันธ์ข่าวการรับสมัครงานสำหรับแฟน...,{'data': [{'created_time': '2018-09-05T10:47:0...


In [0]:
def getMIDfromDF(x):
  return x.split('_')[1]

In [0]:
df['mid'] = df.id.apply(getMIDfromDF)

In [0]:
if len(df[df.comments.isna() == False]) > 0:
  for i in  df[df.comments.isna() == False].comments.values:
    text = i['data']
    dftmp_comments = pd.DataFrame(i['data'],columns=['id','from','message'])
    df_comments = pd.concat([df_comments,dftmp_comments],ignore_index=True)

In [11]:
df_comments

,id,from,message
0,10157760612950477_10157760628495477,NaN,มีให้ฟังย้อนหลังไหมครับ
1,10157716613265477_10157717656375477,NaN,
2,10157578214210477_10157579595310477,NaN,เยี่ยมมาก
3,760219634397284_760237327728848,"{'name': 'Softnix', 'id': '162423635476'}",ใครมีคำถามสามารถฝากมาได้เลยครับ
4,760219634397284_760257901060124,NaN,อยากทราบตัวอย่าง used cases ต่าง ๆ ของ anomaly...
...,...,...,...
64,2116637725047263_2116639891713713,NaN,😍
65,2116637725047263_2126147057429663,NaN,👍
66,2116637725047263_2116646745046361,NaN,เสียงขาดครับ
67,2116637725047263_2119988848045484,NaN,ทักทาย


In [0]:
def getMID(x):
  return x.split('_')[0]
def getCID(x):
  return x.split('_')[1]
def getUserComment(x):
  if x != "":
    return str(x['name'])
  return ""

In [0]:
df_comments['mid'] = df_comments.id.apply(getMID)
df_comments['cid'] = df_comments.id.apply(getCID)
#df_comments['cname'] = df_comments['from'].apply(getUserComment)

In [0]:
df_result = pd.merge(df_comments,
                 df[['id','created_time','message','mid']],
                 on='mid', 
                 how='left')

In [25]:
df_result

,id_x,from,message_x,mid,cid,id_y,created_time,message_y
0,10157760612950477_10157760628495477,NaN,มีให้ฟังย้อนหลังไหมครับ,10157760612950477,10157760628495477,162423635476_10157760612950477,2019-11-26T12:39:35+0000,ผ่านไปแล้วกับงาน CDIC 2019 วันแรก พรุ่งนี้ยังม...
1,10157716613265477_10157717656375477,NaN,,10157716613265477,10157717656375477,162423635476_10157716613265477,2019-11-12T03:01:31+0000,งานสัมมนา Show Khong Day on Stage เช้านี้มาอัป...
2,10157578214210477_10157579595310477,NaN,เยี่ยมมาก,10157578214210477,10157579595310477,162423635476_10157578214210477,2019-09-29T00:45:08+0000,Softnix เรามีโครงการสำหรับนักศึกษาฝึกงาน เรามี...
3,760219634397284_760237327728848,"{'name': 'Softnix', 'id': '162423635476'}",ใครมีคำถามสามารถฝากมาได้เลยครับ,760219634397284,760237327728848,162423635476_760219634397284,2019-09-27T06:56:50+0000,"""โชว์ของเดย์"" เรื่อง Anomaly Detection"
4,760219634397284_760257901060124,NaN,อยากทราบตัวอย่าง used cases ต่าง ๆ ของ anomaly...,760219634397284,760257901060124,162423635476_760219634397284,2019-09-27T06:56:50+0000,"""โชว์ของเดย์"" เรื่อง Anomaly Detection"
...,...,...,...,...,...,...,...,...
64,2116637725047263_2116639891713713,NaN,😍,2116637725047263,2116639891713713,162423635476_2116637725047263,2018-09-18T06:54:58+0000,Data Governance กับ โครงการ Big Data มันสำคัญอ...
65,2116637725047263_2126147057429663,NaN,👍,2116637725047263,2126147057429663,162423635476_2116637725047263,2018-09-18T06:54:58+0000,Data Governance กับ โครงการ Big Data มันสำคัญอ...
66,2116637725047263_2116646745046361,NaN,เสียงขาดครับ,2116637725047263,2116646745046361,162423635476_2116637725047263,2018-09-18T06:54:58+0000,Data Governance กับ โครงการ Big Data มันสำคัญอ...
67,2116637725047263_2119988848045484,NaN,ทักทาย,2116637725047263,2119988848045484,162423635476_2116637725047263,2018-09-18T06:54:58+0000,Data Governance กับ โครงการ Big Data มันสำคัญอ...


In [0]:
df_result.drop(['from'],axis=1).to_csv('./result.csv')

In [28]:
!pip install pythainlp==1.7

     |████████████████████████████████| 10.3MB 2.7MB/s 
     |████████████████████████████████| 276kB 43.8MB/s 
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp36-cp36m-linux_x86_64.whl size=862449 sha256=b68aa9b728884ec8f2b71fc93bb5f4a794793c7384179548934d63f421690c0d
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
Successfully built marisa-trie


In [29]:
import pythainlp

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.


In [33]:
pythainlp.sentiment("เช้านี้อากาศสดใส")

'pos'